In [64]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# 1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [32]:
# Start the Selenium WebDriver
driver = webdriver.Chrome()  # Make sure you have chromedriver installed and in the system path


# Ask user for the product to search
product_to_search = input("Enter the product to search: ")

# Open Amazon.in
driver.get("https://www.amazon.in/")

# Find the search box and enter the product to search
search_box = driver.find_element(By.ID, "twotabsearchtextbox")
search_box.send_keys(product_to_search)
search_box.send_keys(Keys.RETURN)

# Wait for the search results page to load
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-index='1']")))

# Find all the product containers on the page
product_containers = driver.find_elements(By.CSS_SELECTOR, 'div[data-component-type="s-search-result"]')

base_url = "https://www.amazon.in"


search_url = f"{base_url}/s?k={product_to_search.replace(' ', '+')}"

print(search_url)
print()

for container in product_containers:
    try:
        product_title_element = container.find_element(By.CSS_SELECTOR, "span.a-text-normal")
        
        product_title = product_title_element.get_attribute("innerHTML").strip()
        
        product_url_element = container.find_element(By.CSS_SELECTOR, "a.a-link-normal.a-text-normal")
        
        product_url = product_url_element.get_attribute("href")
        
        print("Title:", product_title)
        
        print("URL:", product_url)
        
        print("-" * 50)
        
    except:
        continue
    


Enter the product to search: guitar
https://www.amazon.in/s?k=guitar

Title: Kadence Frontier Jumbo Semi Acoustic Guitar With Die Cast Keys, Super Combo (Bag, 1 pack Strings, Strap, Picks, Capo (Black)
URL: https://www.amazon.in/sspa/click?ie=UTF8&spc=MToxNDAwNTc2NTgxMzU3NDY5OjE2ODcyNDk0ODU6c3BfYXRmOjIwMTIxNzg0MjQwOTk4OjowOjo&url=%2FKadence-Frontier-Acoustic-Guitar-Equalizer%2Fdp%2FB01HRIHJWY%2Fref%3Dsr_1_1_sspa%3Fcrid%3D324VDB7SVM0S%26keywords%3Dguitar%26qid%3D1687249485%26sprefix%3D%252Caps%252C265%26sr%3D8-1-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1
--------------------------------------------------
Title: Kadence Frontier guitar with Online Guitar learning course , Wine Red Acoustic Guitar with Die Cast Keys, Set of Strings, Strap, Picks and Bag (Wine Red, Acoustic)
URL: https://www.amazon.in/sspa/click?ie=UTF8&spc=MToxNDAwNTc2NTgxMzU3NDY5OjE2ODcyNDk0ODU6c3BfYXRmOjIwMDk2NjA4NzUwNDk4OjowOjo&url=%2FKadence-Frontier-Acoustic-Guitar-Strings%2Fdp%2FB01GDZ46AA%2Fref%3Dsr_1_2_ssp

# 2.In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand- Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [34]:
# Wait for the search results page to load
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-index='1']")))

# Scrape details from multiple pages
page_count = 0
products_data = []
while page_count < 3:
    # Find all the product containers on the page
    product_containers = driver.find_elements(By.CSS_SELECTOR, 'div[data-component-type="s-search-result"]')

    # Iterate over each product container and scrape the details
    for container in product_containers:
        product_data = {}
        try:
            product_title_element = container.find_element(By.CSS_SELECTOR, "span.a-text-normal")
            product_title = product_title_element.get_attribute("innerHTML").strip()
        except:
            product_title = "-"
        try:
            product_brand_element = container.find_element(By.CSS_SELECTOR, "span.a-text-normal")
            product_brand = product_brand_element.get_attribute("innerHTML").split()[0]
        except:
            product_brand = "-"
        try:
            product_url_element = container.find_element(By.CSS_SELECTOR, "a.a-link-normal.a-text-normal")
            product_url = product_url_element.get_attribute("href")
        except:
            product_url = "-"
        try:
            product_price_element = container.find_element(By.CSS_SELECTOR, "span.a-price-whole")
            product_price = product_price_element.get_attribute("innerHTML").strip()
        except:
            product_price = "-"
        try:
            product_return_element = container.find_element(By.CSS_SELECTOR, "div.a-row.a-size-small span.a-truncate")
            product_return = product_return_element.get_attribute("innerHTML").strip()
        except:
            product_return = "-"
        try:
            product_delivery_element = container.find_element(By.XPATH,".//span[@class='a-color-base a-text-bold']")
            product_delivery = product_delivery_element.get_attribute("innerHTML").strip()
        except:
            product_delivery="-"
        try:
            product_avail_element = container.find_element(By.XPATH, './/span[@class="a-size-base a-color-price"]')
            product_avail = product_avail_element.get_attribute("innerHTML").strip()
        except:
            product_avail = "-"
        
        # Store the scraped data in a dictionary
        product_data["Brand Name"] = product_brand
        product_data["Name of the Product"] = product_title
        product_data["Price"] = product_price
        product_data["Return/Exchange"] = product_return
        product_data["Expected Delivery"] = product_delivery
        product_data["Availability"] = product_avail
        product_data["Product URL"] = product_url
        
        products_data.append(product_data)

    # Check if there is a "Next" button to navigate to the next page
    next_button = driver.find_element(By.LINK_TEXT,"Next")
    if next_button.get_attribute("aria-disabled") == "true":
        break
    else:
        next_button.click()
        page_count += 1
        time.sleep(2)  # Add a small delay between pages

# Close the browser
driver.quit()

pd.set_option('display.max_colwidth', None)

# Create a DataFrame from the scraped data
df = pd.DataFrame(products_data)


# Save the DataFrame to a CSV file
df.to_csv("product_details.csv", index=False)

# Display the DataFrame
print(df)


     Brand Name  \
0           ESH   
1           ESH   
2           ESH   
3        GUITAR   
4        JUAREZ   
..          ...   
159      Fender   
160      Fender   
161  Tanglewood   
162  A,Acoustio   
163       Vault   

                                                                                                                                                       Name of the Product  \
0                                    ESH Creations E41CHEQ Jumbo High Gloss Semi Acoustic Guitar with Foam Padded Bag, Tuner, Pro Capo, Strings Set, Belt, 3 Picks (Brown)   
1                                        ESH Creations E40CHEQ High Gloss Semi Acoustic Guitar with Foam Padded Bag, Tuner, Pro Capo, Strings Set, Belt, 3 Picks (Natural)   
2                                        ESH Creations E39CHEQ High Gloss Semi Acoustic Guitar with Foam Padded Bag, Tuner, Pro Capo, Strings Set, Belt, 3 Picks (Natural)   
3                                      GUITAR BRO - COMBO (Black Acoustic G

In [35]:
df2 = pd.read_csv('product_details.csv')
df2


def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

## display dataframe
df2.style.format({'Product URL': make_clickable})

,Brand Name,Name of the Product,Price,Return/Exchange,Expected Delivery,Availability,Product URL
0,ESH,"ESH Creations E41CHEQ Jumbo High Gloss Semi Acoustic Guitar with Foam Padded Bag, Tuner, Pro Capo, Strings Set, Belt, 3 Picks (Brown)","6,497",-,"Tuesday, 27 June",-,https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo2NDcwNDQzNDc2MDExNjU3OjE2ODcyNDk1NDA6c3BfYXRmX25leHQ6MjAxMTQxMTI4MjgxOTg6OjA6Og&url=%2FESH-Creations-E41CHEQ-Acoustic-Strings%2Fdp%2FB0BMGG1Q21%2Fref%3Dsr_1_49_sspa%3Fcrid%3D324VDB7SVM0S%26keywords%3Dguitar%26qid%3D1687249540%26sprefix%3D%252Caps%252C265%26sr%3D8-49-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1
1,ESH,"ESH Creations E40CHEQ High Gloss Semi Acoustic Guitar with Foam Padded Bag, Tuner, Pro Capo, Strings Set, Belt, 3 Picks (Natural)","5,897",-,"Tuesday, 27 June",-,https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo2NDcwNDQzNDc2MDExNjU3OjE2ODcyNDk1NDA6c3BfYXRmX25leHQ6MjAxMTQxMTI4Mjg1OTg6OjA6Og&url=%2FESH-Creations-E40CHEQ-Acoustic-Strings%2Fdp%2FB0BMG7BB82%2Fref%3Dsr_1_50_sspa%3Fcrid%3D324VDB7SVM0S%26keywords%3Dguitar%26qid%3D1687249540%26sprefix%3D%252Caps%252C265%26sr%3D8-50-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1
2,ESH,"ESH Creations E39CHEQ High Gloss Semi Acoustic Guitar with Foam Padded Bag, Tuner, Pro Capo, Strings Set, Belt, 3 Picks (Natural)","5,397",-,"Monday, 3 July",Only 1 left in stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo2NDcwNDQzNDc2MDExNjU3OjE2ODcyNDk1NDA6c3BfYXRmX25leHQ6MjAxMTQxMTI4Mjc4OTg6OjA6Og&url=%2FESH-Creations-E39CHEQ-Acoustic-Strings%2Fdp%2FB0BMVQLWF7%2Fref%3Dsr_1_51_sspa%3Fcrid%3D324VDB7SVM0S%26keywords%3Dguitar%26qid%3D1687249540%26sprefix%3D%252Caps%252C265%26sr%3D8-51-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1
3,GUITAR,"GUITAR BRO - COMBO (Black Acoustic Guitar for Beginner, Guitar Learning Kit, Song Book, Video Lectures, Accessories like Pick, Bag)","8,999",-,"Thursday, 22 June",-,https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo2NDcwNDQzNDc2MDExNjU3OjE2ODcyNDk1NDA6c3BfYXRmX25leHQ6MjAwMzE2ODgyMjYyMDE6OjA6Og&url=%2FGUITAR-BRO-Acoustic-Beginner-Accessories%2Fdp%2FB07T84GC8L%2Fref%3Dsr_1_52_sspa%3Fcrid%3D324VDB7SVM0S%26keywords%3Dguitar%26qid%3D1687249540%26sprefix%3D%252Caps%252C265%26sr%3D8-52-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1
4,JUAREZ,JUAREZ Acoustic Guitar 3xE 1st String & 2 Pick Set JRGSE32,75,-,"Tomorrow, 21 June",-,https://www.amazon.in/JUAREZ-Acoustic-Guitar-String-JRGSE32/dp/B08MW1WPW5/ref=sr_1_53?crid=324VDB7SVM0S&keywords=guitar&qid=1687249540&sprefix=%2Caps%2C265&sr=8-53
5,Kadence,Kadence Concert Sized Ukulele 24inch (Built in Equalizer) With Bag and Tuner (Saple Wood With Binding),"3,999",-,"Thursday, 22 June",-,https://www.amazon.in/Kadence-Concert-Ukulele-Equalizer-Binding/dp/B08PNSGQBL/ref=sr_1_54?crid=324VDB7SVM0S&keywords=guitar&qid=1687249540&sprefix=%2Caps%2C265&sr=8-54
6,Yamaha,"Yamaha FS100C Acoustic Guitar, Black","9,879",-,"Saturday, 24 June",-,https://www.amazon.in/Yamaha-FS-100C-6-String-Acoustic-without/dp/B00PAS9CK6/ref=sr_1_55?crid=324VDB7SVM0S&keywords=guitar&qid=1687249540&sprefix=%2Caps%2C265&sr=8-55
7,Intern,Intern INT-38C Acoustic Guitar Kit (Red),"2,345",-,"Tomorrow, 21 June",-,https://www.amazon.in/Intern-INT-38C-Acoustic-Guitar-Kit/dp/B073B6TF31/ref=sr_1_56?crid=324VDB7SVM0S&keywords=guitar&qid=1687249540&sprefix=%2Caps%2C265&sr=8-56
8,INTERN,"INTERN Cutaway Design Acoustic Guitar Pack -Humidity Proof, Bend resistance, Durable Action, Natural tone & bright resonance. Black Carbon Fibre/Fiber Guitar with Bag, Strap, Strings set & Plectrums.","2,599",-,"Thursday, 22 June",Only 1 left in stock.,https://www.amazon.in/Acoustic-Humidity-resistance-resonance-Plectrums/dp/B09RQZ6HM2/ref=sr_1_57?crid=324VDB7SVM0S&keywords=guitar&qid=1687249540&sprefix=%2Caps%2C265&sr=8-57
9,Juârez,"Juârez JRZ38C/ZEB 6 Strings Acoustic Guitar 38 Inch Cutaway, Right Handed, Zebrawood with Bag, Strings, Picks and Strap","2,944",-,"Tomorrow, 21 June",-,https://www.amazon.in/Juarez-Strings-Acous

# 3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [47]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


    # Start the web driver
driver = webdriver.Chrome()  # You can change this to the appropriate web driver for your browser

    # Open Google Images


def scrape_images(keyword):
    
    driver.get("https://www.google.com/imghp")
    
    # Find the search bar element
    wait = WebDriverWait(driver, 20)
    search_bar = driver.find_element(By.NAME, "q")

        # Clear the search bar and enter the keyword
    search_bar.clear()
    search_bar.send_keys(keyword)
    search_bar.send_keys(Keys.RETURN)

        # Wait for the search results to load
    time.sleep(3)

        # Scroll to load more images
    for _ in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        # Find the image elements
    image_container = driver.find_element(By.CSS_SELECTOR,"div.islrc")
        
    image_elements = image_container.find_elements(By.XPATH,'.//img[@jsname="Q4LuWd"]')
        
        # Scrape the image URLs
    image_urls = []
    for image in image_elements[:10]:
        src = image.get_attribute('src')
        if src and not src.startswith('data:image'):
            image_urls.append(src)

        # Print the image URLs
    print(f"Images for {keyword}:")
    for url in image_urls:
        print(url)
        

# Scrape images for the given keywords
keywords = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']
for keyword in keywords:
    scrape_images(keyword)
    print()
driver.quit()

Images for fruits:

Images for cars:

Images for Machine Learning:

Images for Guitar:

Images for Cakes:



# 4 .Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd



driver = webdriver.Chrome()  # Make sure you have chromedriver installed and in the system path

# Open Amazon.in
driver.get("https://www.flipkart.com/")
# close login if popup appears

try:
    close_button = driver.find_element(By.XPATH,'//button[@class="_2KpZ6l _2doB4z"]')
    close_button.click()
except:
    pass


# Find the search box and enter the product to search
search_box = driver.find_element(By.NAME, "q")
search_box.send_keys("Smartphones")
search_box.send_keys(Keys.RETURN)

# Wait for the search results page to load
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[class='_1YokD2 _3Mn1Gg']")))

base_url = 'https://www.flipkart.com'
search_url = f'{base_url}/search?q={"smartphones"}'

    # Send a GET request to the search URL
response = requests.get(search_url)
soup = BeautifulSoup(response.content, 'html.parser')

    # Extract the required details from the search results
products = soup.find_all('div', {'class': '_2kHMtA'})

smartphone_list = []
for product in products:
    try:
        brand = product.find('div', {'class': '_4rR01T'}).text.split()[0]
    except AttributeError:
        brand = '-'
    try:
        name = product.find('div', {'class': '_4rR01T'}).text
    except AttributeError:
        name = '-'

    try:
        color_el = product.find('div', {'class': '_4rR01T'}).text
        color = color_el[color_el.index("(") + 1: color_el.index(")")]
    except AttributeError:
        color = '-'

    try:
        ram = product.find('ul', {'class': '_1xgFaf'}).find_all('li')[0].text.split("|")[0].strip()
    except AttributeError:
        ram = '-'

    try:
        storage = product.find('ul', {'class': '_1xgFaf'}).find_all('li')[0].text.split("|")[1].strip()
    except AttributeError:
        storage = '-'

    try:
        primary_camera = product.find('ul', {'class': '_1xgFaf'}).find_all('li')[2].text.split("|")[0].strip()
    except AttributeError:
        primary_camera = '-'

    try:
        secondary_camera = product.find('ul', {'class': '_1xgFaf'}).find_all('li')[2].text.split("|")[1].strip()
    except AttributeError:
        secondary_camera = '-'

    try:
        display_size = product.find('ul', {'class': '_1xgFaf'}).find_all('li')[1].text
    except AttributeError:
        display_size = '-'

    try:
        battery_capacity = product.find('ul', {'class': '_1xgFaf'}).find_all('li')[3].text
        
    except (AttributeError, IndexError):
        battery_capacity = '-'

    try:
        price = product.find('div', {'class': '_30jeq3 _1_WHN1'}).text
    except AttributeError:
        price = '-'

    try:

        product_url = product.find('a', {'class': '_1fQZEK'})['href']
        
    except (AttributeError, TypeError):
        product_url = '-'

    smartphone = {
        'Brand Name': brand,
        'Smartphone Name': name,
        'Colour': color,
        'RAM': ram,
        'Storage(ROM)': storage,
        'Primary Camera': primary_camera,
        'Secondary Camera': secondary_camera,
        'Display Size': display_size,
        'Battery Capacity': battery_capacity,
        'Price': price,
        'Product URL': product_url
    }

        
    smartphone_list.append(smartphone)


pd.set_option('display.max_colwidth', None)

    
df = pd.DataFrame(smartphone_list)
    
df.to_csv("smartphones.csv", index=False)

flipkart_smartphone = pd.read_csv('smartphones.csv')

def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

## display dataframe
flipkart_smartphone.style.format({'Product URL': make_clickable})

driver.quit()

,Brand Name,Smartphone Name,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Battery Capacity,Price,Product URL
0,Infinix,"Infinix SMART 7 (Azure Blue, 64 GB)","Azure Blue, 64 GB",4 GB RAM,64 GB ROM,13MP + AI Lens,5MP Front Camera,16.76 cm (6.6 inch) HD+ Display,6000 mAh Battery,"₹7,299",/infinix-smart-7-azure-blue-64-gb/p/itm7b72b66485a59?pid=MOBGMTW2WYG2A2FM&lid=LSTMOBGMTW2WYG2A2FMSRLXTO&marketplace=FLIPKART&q=smartphones&store=tyy%2F4io&srno=s_1_1&otracker=search&fm=organic&iid=78f115a3-85fe-48be-a052-3320f29e5460.MOBGMTW2WYG2A2FM.SEARCH&ppt=None&ppn=None&ssid=94mrpwaxb40000001687256256701&qH=6ea4465d0add4685
1,Infinix,"Infinix SMART 7 (Night Black, 64 GB)","Night Black, 64 GB",4 GB RAM,64 GB ROM,13MP + AI Lens,5MP Front Camera,16.76 cm (6.6 inch) HD+ Display,6000 mAh Battery,"₹7,299",/infinix-smart-7-night-black-64-gb/p/itm7b72b66485a59?pid=MOBGMTW2T7W2UPB9&lid=LSTMOBGMTW2T7W2UPB9VAZZSU&marketplace=FLIPKART&q=smartphones&store=tyy%2F4io&srno=s_1_2&otracker=search&fm=organic&iid=78f115a3-85fe-48be-a052-3320f29e5460.MOBGMTW2T7W2UPB9.SEARCH&ppt=None&ppn=None&ssid=94mrpwaxb40000001687256256701&qH=6ea4465d0add4685
2,Infinix,"Infinix SMART 7 (Emerald Green, 64 GB)","Emerald Green, 64 GB",4 GB RAM,64 GB ROM,13MP + AI Lens,5MP Front Camera,16.76 cm (6.6 inch) HD+ Display,6000 mAh Battery,"₹7,299",/infinix-smart-7-emerald-green-64-gb/p/itm7b72b66485a59?pid=MOBGMTW2HFCMQY35&lid=LSTMOBGMTW2HFCMQY35R7FJQZ&marketplace=FLIPKART&q=smartphones&store=tyy%2F4io&srno=s_1_3&otracker=search&fm=organic&iid=78f115a3-85fe-48be-a052-3320f29e5460.MOBGMTW2HFCMQY35.SEARCH&ppt=None&ppn=None&ssid=94mrpwaxb40000001687256256701&qH=6ea4465d0add4685
3,Infinix,"Infinix HOT 30i (Mirror Black, 64 GB)","Mirror Black, 64 GB",4 GB RAM,64 GB ROM,50MP + AI Lens,5MP Front Camera,16.76 cm (6.6 inch) HD+ Display,5000 mAh Battery,"₹7,999",/infinix-hot-30i-mirror-black-64-gb/p/itm8b42a43b35895?pid=MOBGNPK2QRUEBEHG&lid=LSTMOBGNPK2QRUEBEHGKBH4RF&marketplace=FLIPKART&q=smartphones&store=tyy%2F4io&srno=s_1_4&otracker=search&fm=organic&iid=78f115a3-85fe-48be-a052-3320f29e5460.MOBGNPK2QRUEBEHG.SEARCH&ppt=None&ppn=None&ssid=94mrpwaxb40000001687256256701&qH=6ea4465d0add4685
4,Infinix,"Infinix HOT 30i (Glacier Blue, 64 GB)","Glacier Blue, 64 GB",4 GB RAM,64 GB ROM,50MP + AI Lens,5MP Front Camera,16.76 cm (6.6 inch) HD+ Display,5000 mAh Battery,"₹7,999",/infinix-hot-30i-glacier-blue-64-gb/p/itm8b42a43b35895?pid=MOBGNPK2BBNPWJYR&lid=LSTMOBGNPK2BBNPWJYRIQDVWM&marketplace=FLIPKART&q=smartphones&store=tyy%2F4io&srno=s_1_5&otracker=search&fm=organic&iid=78f115a3-85fe-48be-a052-3320f29e5460.MOBGNPK2BBNPWJYR.SEARCH&ppt=None&ppn=None&ssid=94mrpwaxb40000001687256256701&qH=6ea4465d0add4685
5,Infinix,"Infinix HOT 30i (Glacier Blue, 128 GB)","Glacier Blue, 128 GB",8 GB RAM,128 GB ROM,50MP + AI Lens,5MP Front Camera,16.76 cm (6.6 inch) HD+ Display,5000 mAh Battery,"₹8,999",/infinix-hot-30i-glacier-blue-128-gb/p/itm8b42a43b35895?pid=MOBGNPK2ZTGZBRXE&lid=LSTMOBGNPK2ZTGZBRXEAFQ5YT&marketplace=FLIPKART&q=smartphones&store=tyy%2F4io&spotlightTagId=BestsellerId_tyy%2F4io&srno=s_1_6&otracker=search&fm=organic&iid=78f115a3-85fe-48be-a052-3320f29e5460.MOBGNPK2ZTGZBRXE.SEARCH&ppt=None&ppn=None&ssid=94mrpwaxb40000001687256256701&qH=6ea4465d0add4685
6,Infinix,"Infinix HOT 30i (Diamond White, 128 GB)","Diamond White, 128 GB",8 GB RAM,128 GB ROM,50MP + AI Lens,5MP Front Camera,16.76 cm (6.6 inch) HD+ Display,5000 mAh Battery,"₹8,999",/infinix-hot-30i-diamond-white-128-gb/p/itm8b42a43b35895?pid=MOBGNPK29ZWFHGGD&lid=LSTMOBGNPK29ZWFHGGDKTDNDX&marketplace=FLIPKART&q=smartphones&store=tyy%2F4io&srno=s_1_7&otracker=search&fm=organic&iid=78f115a3-85fe-48be-a052-3320f29e5460.MOBGNPK29ZWFHGGD.SEARCH&ppt=None&ppn=None&ssid=94mrpwaxb40000001687256256701&qH=6ea4465d0add4685
7,Infinix,"Infinix HOT 30i (Mirror Black, 128 GB)","Mirror Black, 128 GB",8 GB RAM,128 GB ROM,50MP + AI Lens,5MP Front Camera,16.76 cm (6.6 inch) HD+ Display,5000 mAh Battery,"₹8,999",/infinix-hot-30i-mirror-black-128-gb/p/itm8

# 5 . Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [36]:
!pip install geopy

     |████████████████████████████████| 119 kB 688 kB/s eta 0:00:01
     |████████████████████████████████| 40 kB 1.3 MB/s eta 0:00:011


In [37]:
from geopy.geocoders import Nominatim

def get_coordinates(city):
    geolocator = Nominatim(user_agent="my_app")  # Create a geocoder instance

    location = geolocator.geocode(city)  # Geocode the city
    if location:
        latitude = location.latitude
        longitude = location.longitude
        return latitude, longitude
    else:
        return None

# Get the city from the user
city = input("Enter the city to search: ")

# Get the coordinates
coordinates = get_coordinates(city)

if coordinates:
    latitude, longitude = coordinates
    print("Latitude:", latitude)
    print("Longitude:", longitude)
else:
    print("Coordinates not found for the specified city.")


Enter the city to search: delhi
Latitude: 28.6273928
Longitude: 77.1716954


# 6.Write a program to scrap all the available details of best gaming laptops from digit.in.

In [93]:
import requests
from bs4 import BeautifulSoup

def scrape_digit_gaming_laptops():
    url = 'https://www.digit.in/top-products/best-gaming-laptops-40.html'

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    laptops = []

    laptop_list = soup.find_all('div', class_='TopNumbeHeading')

    for item in laptop_list:
        name = item.find('h3').text.strip()
        specs_list = item.find('div', class_='Specs-Wrap')
        if specs_list:
            specs = [spec.text.strip() for spec in specs_list.find_all('li')]
        else:
            specs = []

        laptop = {
            'Name': name,
            'Specifications': specs
        }

        laptops.append(laptop)

    return laptops

# Usage
results = scrape_digit_gaming_laptops()

# Print the scraped data
for laptop in results:
    print(laptop)


{'Name': 'HP OMEN 17 (2023)', 'Specifications': []}
{'Name': 'MSI Titan GT77 12UHS', 'Specifications': []}
{'Name': 'Lenovo Legion 5i Pro', 'Specifications': []}
{'Name': 'ASUS ROG Strix Scar 18 2023', 'Specifications': []}
{'Name': 'Acer Predator Helios Neo 16', 'Specifications': []}
{'Name': 'ASUS ROG Zephyrus G14', 'Specifications': []}
{'Name': 'MSI Cyborg 15', 'Specifications': []}


# 7.Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”/ “Industry”.

In [55]:
from bs4 import BeautifulSoup
import requests

url = 'https://www.forbes.com/billionaires/'


In [60]:
response = requests.get(url)

response

<Response [200]>

In [61]:
soup = BeautifulSoup(response.content)
print(soup)


<!DOCTYPE html>
<html><head><meta charset="utf-8"/><meta content="#000000" name="theme-color"/><title>Forbes Billionaires 2023: The Richest People In The World</title><link href="https://i.forbesimg.com/favicon.ico" rel="shortcut icon"/><link href="https://www.forbes.com/billionaires/" rel="canonical"/><meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=yes" name="viewport"/><meta content="origin-when-cross-origin" name="referrer"/><meta content="Forbes presents the 2023 World's Billionaires List. View the richest people in the world including the youngest billionaires, female billionaires and newest billionaires." itemprop="description" name="description"/><meta content="billionaires, the world's billionaires, how many billionaires are there in the world, how many billionaires in the us, how many billionaires are there, wealth, Jeff Bezos, Kylie Jenner, Elon Musk, Kylie, Steward, Bezos, Laurene Powell Jobs, Donald Trump, richest people in the world, world’s

In [40]:
import requests
from bs4 import BeautifulSoup

# Send a GET request to the Forbes billionaires page
url = 'https://www.forbes.com/billionaires/'
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

print(soup)

# Find the table containing the billionaires' details

table = soup.find('div',class_='Table_tableBlock__rRFM8 Table_billionaires__qIfS_')

print(table)

# Scrape the details for each billionaire
billionaires = table.find_all('div', role_='row')


# Iterate over each billionaire and extract the required details
for billionaire in billionaires:
    print('OK')
    rank = billionaire.find('div', class_='Table_rank___YBhk Table_dataCell__2QCve').text.strip()
    name = billionaire.find('div', class_='Table_dataCell__2QCve').text.strip()
    net_worth = billionaire.find('div', class_='Table_netWorth___L4R5 Table_dataCell__2QCve').text.strip()
    age = billionaire.find('div', class_='Table_dataCell__2QCve').text.strip()
    citizenship = billionaire.find('div', class_='TableRow_cell__db-hv Table_cell__houv9').text.strip()
    source = billionaire.find('div', class_='Table_dataCell__2QCve').text.strip()
    industry = billionaire.find('div', class_='Table_dataCell__2QCve').text.strip()
    
    # Print the scraped details
    print('Rank:', rank)
    print('Name:', name)
    print('Net Worth:', net_worth)
    print('Age:', age)
    print('Citizenship:', citizenship)
    print('Source:', source)
    print('Industry:', industry)
    print('-' * 50)



<!DOCTYPE html>
<html><head><meta charset="utf-8"/><meta content="#000000" name="theme-color"/><title>Forbes Billionaires 2023: The Richest People In The World</title><link href="https://i.forbesimg.com/favicon.ico" rel="shortcut icon"/><link href="https://www.forbes.com/billionaires/" rel="canonical"/><meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=yes" name="viewport"/><meta content="origin-when-cross-origin" name="referrer"/><meta content="Forbes presents the 2023 World's Billionaires List. View the richest people in the world including the youngest billionaires, female billionaires and newest billionaires." itemprop="description" name="description"/><meta content="billionaires, the world's billionaires, how many billionaires are there in the world, how many billionaires in the us, how many billionaires are there, wealth, Jeff Bezos, Kylie Jenner, Elon Musk, Kylie, Steward, Bezos, Laurene Powell Jobs, Donald Trump, richest people in the world, worldâ

AttributeError: 'NoneType' object has no attribute 'find_all'

# 8.Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [27]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

def get_comments(video_url, max_comments=500):
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run Chrome in headless mode
    service = Service('path/to/chromedriver')  # Replace with actual path to chromedriver executable
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    # Open the YouTube video URL
    driver.get(video_url)
    time.sleep(5)  # Wait for the page to load
    
    # Scroll down to load comments dynamically
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    while len(driver.find_elements(By.CSS_SELECTOR, "#contents #content-text")) < max_comments:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    # Extract comment elements
    comment_elements = driver.find_elements(By.CSS_SELECTOR, "#contents #content-text")
    upvote_elements = driver.find_elements(By.CSS_SELECTOR, "#contents #vote-count-middle")
    time_elements = driver.find_elements(By.CSS_SELECTOR, "#contents #header-author > yt-formatted-string > a")
    
    # Extract comment details
    comments = []
    for i in range(len(comment_elements)):
        comment_text = comment_elements[i].text.strip()
        upvote_count = upvote_elements[i].text.strip()
        time_text = time_elements[i].get_attribute("innerText").strip()
        comments.append({
            "comment": comment_text,
            "upvotes": upvote_count,
            "time": time_text
        })
        if len(comments) >= max_comments:
            break
    
    # Close the browser
    driver.quit()
    
    return comments

# Ask user for input
video_url = input("Enter the YouTube video URL: ")
comments = get_comments(video_url, max_comments=500)

# Print the extracted comments, upvotes, and time
for comment in comments:
    print(f"Comment: {comment['comment']}")
    print(f"Upvotes: {comment['upvotes']}")
    print(f"Time: {comment['time']}")
    print('-' * 50)



Enter the YouTube video URL: https://www.youtube.com/watch?v=mih_xPmo-Tc
Comment: Thank you Sir for the effective content and explanation. I have learnt selenium using Python. Now I am capable to do the biggest projects. Once again Thank you Sir.
Upvotes: 1
Time: 2 years ago
--------------------------------------------------
Comment: Hi Pavan,

Do you have any playlist or any documentation available for mobile automation in visualizer tool using TestNG?
Upvotes: 1
Time: 2 years ago
--------------------------------------------------
Comment: sir i have gone through all session of selenium ...pls provide me notes of these session it will be very helpful for me to crack interview
Upvotes: 
Time: 13 days ago
--------------------------------------------------
Comment: U greatly helped me to understand this concept .thank u so much .and sir do u have base  and pojo class video
Upvotes: 
Time: 1 year ago
--------------------------------------------------
Comment: Hi Sir,
if we are storing in 

# 9.Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property descrption

In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# Set up Chrome WebDriver
#chrome_options = Options()
#chrome_options.add_argument("--headless")  # Run Chrome in headless mode
#chrome_service = Service("path/to/chromedriver")  # Replace with the path to your chromedriver executable
driver = webdriver.Chrome()

# Set the URL for London hostels

# Load the page
driver.get("https://www.hostelworld.com")
driver.find_element(By.ID, "search-input-field").send_keys("London")


# Find all the hostel listings on the page
hostels = driver.find_elements(By.CLASS_NAME, "fabresult")

# Iterate over each hostel and extract the required information
for hostel in hostels:
    # Extract hostel name
    name = hostel.find_element(By.CLASS_NAME, "title").text.strip()

    # Extract distance from city centre
    distance = hostel.find_element(By.CLASS_NAME, "description").text.strip()

    # Extract ratings
    ratings = hostel.find_element(By.CLASS_NAME, "score").text.strip()

    # Extract total reviews
    total_reviews = hostel.find_element(By.CLASS_NAME, "reviews").text.strip()

    # Extract overall reviews
    overall_reviews = hostel.find_element(By.CLASS_NAME, "rating").text.strip()

    # Extract privates from price
    privates_from_price = hostel.find_element(By.CLASS_NAME, "price.title-5").text.strip()

    # Extract dorms from price
    dorms_from_price = hostel.find_element(By.CLASS_NAME, "price.title-6").text.strip()

    # Extract facilities
    facilities = [item.text.strip() for item in hostel.find_elements(By.CLASS_NAME, "facilities-label")]

    # Extract property description
    description = hostel.find_element(By.CLASS_NAME, "ratings").find_element(By.XPATH, "./following-sibling::div").text.strip()

    # Print the extracted information
    print("Hostel Name:", name)
    print("Distance from City Centre:", distance)
    print("Ratings:", ratings)
    print("Total Reviews:", total_reviews)
    print("Overall Reviews:", overall_reviews)
    print("Privates from Price:", privates_from_price)
    print("Dorms from Price:", dorms_from_price)
    print("Facilities:", facilities)
    print("Property Description:", description)
    print("--------------------------------------")

# Quit the driver and close the browser
#driver.quit()


Timed out waiting for page to load


MaxRetryError: HTTPConnectionPool(host='localhost', port=58880): Max retries exceeded with url: /session/986212dab64c81de3fc1883b6fdd24e0/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f78b9a88a00>: Failed to establish a new connection: [Errno 61] Connection refused'))